In [1]:
!pip install tweepy
!pip install networkx
!pip install chart_studio

     |████████████████████████████████| 71kB 2.0MB/s 


In [11]:
import re 
import datetime
import tweepy 
from tweepy import OAuthHandler
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly

In [10]:
class TwitterClient(object): 
    
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = '9ZT84sMENHGuUtGScMrWvIV9D'
        consumer_secret = 'mvQWO17OvQJEONGR5a96KPbqAPpznLPTzFPi1Ke0pEYh2ULK08'
        access_token = '627369688-A6UECJida7K6jggxbo3DNEN4xCuHdrdpBRm6cxHX'
        access_token_secret = 'dbTp5geakYdj0A27OFEEQNwMvRoB72k3McZ5eijXWZEht'
  
        # attempt authentication 

        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True) 
        except: 
            print("Error: Authentication Failed") 
  
    def get_tweets(self, query, count = 200): 
        
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count)
            req_tweets=[]
            startDate = datetime.datetime(2020, 8, 15, 0, 0, 0)
            endDate =   datetime.datetime(2020, 8, 16, 0, 0, 0)
            for t in fetched_tweets:
                if(t.created_at < endDate and t.created_at > startDate):
                  print(t.user.name)
                  req_tweets.append(t)
            follow=[[]]*len(req_tweets)
            followin=[[]]*len(req_tweets)
            for i in range(len(req_tweets)): 
                tweet=req_tweets[i]
                foll=self.api.followers(tweet.user.id)
                
                for f in foll:
                    follow[i].append(f.id)
                folli=self.api.friends(tweet.user.id)
                for f in folli:
                    followin[i].append(f.id)
            G = nx.Graph()
            for i in range(len(req_tweets)):
              G.add_node(req_tweets[i].user.id)
            for i in range(len(req_tweets)):
              tw=req_tweets[i]
              for j in range(i+1,len(req_tweets)):
                z=req_tweets[j]
                if(z.user.id in follow[i]):
                  G.add_edge(tw.user.id,z.user.id)
                if(z.user.id in followin[i]):
                  G.add_edge(z.user.id,tw.user.id)
            # Plotting graph
            chart_studio.tools.set_credentials_file(username='pravash21', api_key='1otDXe8UFaOe2iddV6Ml')
            pos= nx.random_layout(G)
            dmin=1
            ncenter=0
            for n in pos:
              x,y=pos[n]
              d=(x-0.5)**2+(y-0.5)**2
              if d<dmin:
                ncenter=n
                dmin=d
            #p=nx.single_source_shortest_path_length(G,ncenter)
            edge_trace = go.Scatter(
                x=[],
                y=[],
                line=dict(width=0.5,color='#888'),
                hoverinfo='none',
                mode='lines')
            for edge in G.edges():
              x0, y0 = pos[edge[0]]
              x1, y1 = pos[edge[1]]
              edge_trace['x'] += tuple([x0, x1, None])
              edge_trace['y'] += tuple([y0, y1, None])
            node_trace = go.Scatter(
                x=[],
                y=[],
                text=[],
                mode='markers',
                hoverinfo='text',
                marker=dict(
                    showscale=True,
                    colorscale='YlGnBu',
                    reversescale=True,
                    color=[],
                    size=10,
                    colorbar=dict(
                        thickness=15,
                        title='Node Connections',
                        xanchor='left',
                        titleside='right'
                    ),
                    line=dict(width=2)))
            for node in G.nodes():
              x, y = pos[node]
              node_trace['x'] += tuple([x])
              node_trace['y'] += tuple([y])
            for node, adjacencies in enumerate(G.adjacency()):
              node_trace['marker']['color']+=tuple([len(adjacencies[1])])
              node_info = '# of connections: '+str(len(adjacencies[1]))
              node_trace['text']+=tuple([node_info])
            fig = go.Figure(data=[edge_trace, node_trace],
                            layout=go.Layout(
                                title='<br>Twitter Network graph made with Python',
                                titlefont=dict(size=16),
                                showlegend=False,
                                hovermode='closest',
                                margin=dict(b=20,l=5,r=5,t=40),
                                annotations=[ dict(
                                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                                    showarrow=False,
                                    xref="paper", yref="paper",
                                    x=0.005, y=-0.002 ) ],
                                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
            py.plot(fig, filename='networkx')
            
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    
    
    tweets = api.get_tweets(query = '#COVID-19', count = 200) 

if __name__ == "__main__": 
    # calling main function 
    main() 